# Baseline Modeling for NBA Dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from scipy import stats
from random import sample
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
games = pd.read_csv('C:/Users/michael.thabane/Documents/Springboard/Capstone2/nba-games/games.csv')
games_details = pd.read_csv('C:/Users/michael.thabane/Documents/Springboard/Capstone2/nba-games/games_details.csv')
players = pd.read_csv('C:/Users/michael.thabane/Documents/Springboard/Capstone2/nba-games/players.csv')
standings = pd.read_csv('C:/Users/michael.thabane/Documents/Springboard/Capstone2/nba-games/ranking.csv')

In [2]:
final_boxscore = pd.read_csv('C:/Users/michael.thabane/Documents/Springboard/Capstone2/final_boxscore.csv')

In [3]:
unique_players = players.drop(columns=['TEAM_ID', 'SEASON'])
unique_players.drop_duplicates(inplace = True)

In [4]:
unique_players.query('PLAYER_NAME.str.contains("LeBron")', engine='python')

,PLAYER_NAME,PLAYER_ID
199,LeBron James,2544


In [5]:
lebron_stats = final_boxscore.loc[final_boxscore['PLAYER_ID'] == 2544]
lebron_stats

,Unnamed: 0,SEASON,GAME_ID,GAME_DATE_EST,TEAM_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,GAME_LOC,...,FG3AL5,FG3_PCTL5,FTML5,FTAL5,FT_PCTL5,STLL5,PLUS_MINUSL5,TOT_PTSL5,DAYSOFF,YEAR
22,300,2019,21900891,2020-02-29,1610612747,1610612763,1610612747,LAL,Los Angeles,AWAY,...,7.666667,0.353833,4.500000,6.666667,0.597167,0.333333,7.166667,119.833333,1.0,11.0
556,945,2019,21900861,2020-02-25,1610612747,1610612747,1610612740,LAL,Los Angeles,HOME,...,7.166667,0.298833,4.500000,6.500000,0.680500,0.666667,4.500000,118.666667,4.0,11.0
1042,1446,2019,21900842,2020-02-23,1610612747,1610612747,1610612738,LAL,Los Angeles,HOME,...,7.000000,0.343333,4.333333,6.000000,0.694500,0.833333,8.000000,121.166667,2.0,11.0
1359,1772,2019,21900833,2020-02-21,1610612747,1610612747,1610612763,LAL,Los Angeles,HOME,...,7.000000,0.343333,2.833333,4.500000,0.652833,1.166667,12.500000,123.166667,2.0,11.0
1752,2174,2019,21900817,2020-02-12,1610612747,1610612743,1610612747,LAL,Los Angeles,AWAY,...,7.166667,0.382000,2.833333,4.500000,0.652833,1.166667,13.000000,123.000000,9.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566396,575784,2003,20300038,2003-11-01,1610612739,1610612757,1610612739,CLE,Cleveland,AWAY,...,2.000000,0.050000,1.250000,2.500000,0.226000,2.000000,-6.000000,90.250000,4.0,NaN
566659,576047,2003,20300018,2003-10-30,1610612739,1610612756,1610612739,CLE,Cleveland,AWAY,...,1.000000,0.000000,0.333333,1.000000,0.111000,2.333333,-9.000000,91.666667,2.0,NaN
566890,576278,2003,20300014,2003-10-29,1610612739,1610612758,1610612739,CLE,Cleveland,AWAY,...,0.500000,0.000000,0.000000,0.000000,0.000000,1.500000,NaN,91.500000,1.0,NaN
567040,576429,2003,10300011,2003-10-08,1610612739,1610612737,1610612739,CLE,Cleveland,AWAY,...,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,NaN,100.000000,21.0,NaN


In [6]:
lebron_test = lebron_stats.loc[lebron_stats['SEASON'] == 2018]
lebron_train = lebron_stats.iloc[120:]

In [7]:
lebron_train_drop = lebron_train.dropna()
y = lebron_train_drop['PTS']
X = lebron_train_drop[['PTSL5','PTSL3','MINL5','FGML5','FGAL5','FG_PCTL5','FG3ML5','FG3AL5','FG3_PCTL5','FTML5',
                       'FTAL5','FT_PCTL5','STLL5','PLUS_MINUSL5','TOT_PTSL5','DAYSOFF','YEAR']]
X = sm.add_constant(X)
model = sm.OLS(y,X).fit()
model.summary()

C:\Users\michael.thabane\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    PTS   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     2.409
Date:                Sat, 20 Jun 2020   Prob (F-statistic):            0.00152
Time:                        13:30:04   Log-Likelihood:                -2901.4
No. Observations:                 849   AIC:                             5837.
Df Residuals:                     832   BIC:                             5918.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           23.4439     11.235      2.087      0.037       1.392      45.496
PTSL5            0.1329      0.342      0.388      0.698      -0.539       0.805
PTSL3            0.0641      0.117      0.547      0.585      -0.166       0.294
MINL5            0.2082      0.116      1.788      0.074      -0.020       0.437
FGML5            0.9391      0.651      1.442      0.150      -0.339       2.217
FGAL5           -0.5390      0.497     -1.084      0.279      -1.515       0.437
FG_PCTL5       -17.3894     17.022     -1.022      0.307     -50.801      16.022
FG3ML5          -1.9574      1.117     -1.752      0.080      -4.150       0.236
FG3AL5           0.6860      0.531      1.291      0.197      -0.357       1.729
FG3_PCTL5       -2.0303      4.100     -0.495      0.621     -10.077       6.016
FTML5            0.2122      0.753      0.282      0.778      -1.266       1.690
FTAL5           -0.0565      0.635     -0.089      0.929      -1.303       1.190
FT_PCTL5        -6.5451      4.751     -1.378      0.169     -15.871       2.780
STLL5           -0.6383      0.479     -1.332      0.183      -1.579       0.302
PLUS_MINUSL5     0.0509      0.051      0.995      0.320      -0.050       0.151
TOT_PTSL5        0.0621      0.056      1.110      0.267      -0.048       0.172
DAYSOFF         -0.1186      0.186     -0.638      0.523      -0.483       0.246
YEAR            -0.0644      0.149     -0.432      0.666      -0.357       0.228
==============================================================================
Omnibus:                       21.030   Durbin-Watson:                   2.051
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               25.739
Skew:                           0.294   Prob(JB):                     2.58e-06
Kurtosis:                       3.617   Cond. No.                     2.18e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.55e-26. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [8]:
import statsmodels.api as sm
lebron_model_test_drop = lebron_test.dropna()
lebron_model_test_drop = lebron_model_test_drop[['PTSL5','PTSL3','MINL5','FGML5','FGAL5','FG_PCTL5','FG3ML5','FG3AL5','FG3_PCTL5','FTML5',
                       'FTAL5','FT_PCTL5','STLL5','PLUS_MINUSL5','TOT_PTSL5','DAYSOFF','YEAR']]
lebron_model_test_drop = lebron_model_test_drop.assign(const = 1)
cols = lebron_model_test_drop.columns.tolist()
cols = cols[-1:] + cols[:-1]
lebron_model_test_drop = lebron_model_test_drop[cols]
lebron_model_test_drop

,const,PTSL5,PTSL3,MINL5,FGML5,FGAL5,FG_PCTL5,FG3ML5,FG3AL5,FG3_PCTL5,FTML5,FTAL5,FT_PCTL5,STLL5,PLUS_MINUSL5,TOT_PTSL5,DAYSOFF,YEAR
29077,1,30.333333,29.000000,33.319444,11.333333,23.666667,0.481833,1.333333,5.500000,0.257167,6.333333,8.833333,0.700333,1.000000,5.333333,111.333333,3.0,10.0
29396,1,30.666667,30.750000,32.627778,12.000000,23.666667,0.512167,1.166667,5.666667,0.221500,5.500000,8.666667,0.614667,0.833333,3.166667,109.333333,2.0,10.0
29755,1,31.000000,32.000000,33.577778,12.166667,22.500000,0.542167,1.166667,5.500000,0.225500,5.500000,8.666667,0.614667,0.833333,1.666667,109.166667,2.0,10.0
30701,1,30.000000,31.500000,34.538889,12.000000,21.666667,0.551000,1.833333,6.000000,0.299500,4.166667,7.666667,0.545167,0.833333,0.000000,106.833333,5.0,10.0
31320,1,30.333333,31.000000,35.711111,11.666667,21.000000,0.551667,2.166667,6.500000,0.328667,4.833333,8.166667,0.583333,0.833333,-0.666667,110.500000,3.0,10.0
31508,1,29.833333,28.750000,36.363889,11.333333,21.166667,0.533333,2.500000,7.333333,0.319333,4.666667,8.666667,0.512500,0.833333,-1.833333,110.833333,2.0,10.0
32048,1,28.833333,29.000000,38.358333,10.500000,21.166667,0.497167,2.333333,7.166667,0.286000,5.500000,9.166667,0.574333,1.333333,-3.333333,110.500000,3.0,10.0
32513,1,28.166667,29.500000,39.591667,10.166667,20.333333,0.506500,2.500000,7.000000,0.321667,5.333333,8.833333,0.567333,1.833333,-3.666667,113.166667,3.0,10.0
32884,1,28.500000,28.750000,39.213889,10.500000,21.166667,0.502833,2.333333,6.666667,0.293833,5.166667,8.500000,0.572833,1.833333,-0.833333,114.166667,2.0,10.0
33395,1,28.666667,28.750000,39.594444,10.166667,21.000000,0.490167,2.166667,6.833333,0.269833,6.166667,9.166667,0.637667,2.333333,-0.666667,114.833333,2.0,10.0


In [9]:
Y_pred = model.predict(lebron_model_test_drop)
act = lebron_test.dropna()
Y_actual = act['PTS']

In [10]:
RMSE = np.sqrt(mean_squared_error(Y_actual,Y_pred))
r2 = r2_score(Y_actual,Y_pred)

In [11]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [12]:
MAPE = mean_absolute_percentage_error(Y_actual,Y_pred)

In [13]:
print(' RMSE =', RMSE,'\n','r squared = ', r2,'\n','MAPE = ', MAPE)

 RMSE = 7.753315379958686 
 r squared =  -0.021090252249140296 
 MAPE =  25.842116632470884
